# Geschäfte vs Votum

In [1]:
from pathlib import Path
import pandas as pd
import datetime
import utils
import json
import numpy as np
pd.options.plotting.backend = "plotly"

## Load Data

### Load Kantonsrat History

In [2]:
# Load Kantonsrats-member-history
with open(Path('../export/mitglieder.json'), encoding='utf-8') as f:
    kantonsrat = json.load(f)

# Typecast
utils.kantonsrat_to_datetime(kantonsrat)

### Load Geschäfte

In [3]:
df_raw = pd.read_csv(Path('../export/geschaefte.csv'))

# Only Members
df_raw = df_raw[df_raw.erstunterzeichneristkantonsrat == True]

# To Datetime
df_raw['start'] = pd.to_datetime(df_raw['start'])

# Calculate Amtsjahr (Amtsjahr 2020 = 1.5.2020 - 30.4.2021)
df_raw['year'] = df_raw['start'].dt.year
df_raw['month'] = df_raw['start'].dt.month
df_raw['session'] = df_raw.apply(lambda row: row['year'] if row['month'] > 5 else row['year'] - 1, axis=1)

# Concat Name for convenience reasons
df_raw['_name'] = df_raw.apply(lambda row: "%s %s" % (row['erstunterzeichnervorname'], row['erstunterzeichnername']), axis = 1)

df_geschaeft_raw = df_raw.copy()

### Load Voten

In [4]:
df_votum_raw = pd.concat([
    pd.read_csv(Path('../export/votum/votum_0.csv')),
    pd.read_csv(Path('../export/votum/votum_1.csv'))
])

# Remove non members (mostly former members who are now in the Regierungsrat)
df_votum_raw = df_votum_raw[df_votum_raw.ismember == True]

# Typecast
df_votum_raw['sitzung_date'] = pd.to_datetime(df_votum_raw['sitzung_date'])

# Remove empty texts
df_votum_raw = df_votum_raw[df_votum_raw.text.notna()]

# Concat Name for convenience reasons
df_votum_raw['_name'] = df_votum_raw.apply(lambda row: "%s %s" % (row['vorname'], row['name']), axis = 1)

# Calculate Amtsjahr (Amtsjahr 2020 = 1.5.2020 - 30.4.2021)
df_votum_raw['year'] = df_votum_raw['sitzung_date'].dt.year
df_votum_raw['month'] = df_votum_raw['sitzung_date'].dt.month
df_votum_raw['session'] = df_votum_raw.apply(lambda row: row['year'] if row['month'] > 5 else row['year'] - 1, axis=1)

In [5]:
# Cut same date
d = '2002-01-07'
df_votum_raw = df_votum_raw[df_votum_raw.sitzung_date >= d]
df_geschaeft_raw = df_geschaeft_raw[df_geschaeft_raw.eingereicht >= d]

## Calc Difference between Votum and Geschaeft

In [6]:
# Group Geschaeft
df_geschaeft = df_geschaeft_raw.groupby('_name').agg(
    erstunterzeichner = ('titel', 'count'),
    partei = ('erstunterzeichnerpartei', 'first'),
    letztes_geschaeft = ('eingereicht', 'last')
)

# Drop Präsident in Votum
df_votum = df_votum_raw[df_votum_raw.funktion.isna()]
df_votum = df_votum.groupby('_name').agg(
    voten = ('session', 'count'),
    letztes_votum = ('sitzung_date', 'last')
)

# Join!
df = df_votum.join(df_geschaeft)

#Fillna
df = df.fillna(0)

# Calc Abweichung
df['factor'] = 0

df.loc[df.erstunterzeichner > 0, 'factor'] = round(df['erstunterzeichner'] / df['voten'], 3)
df = df.sort_values('factor')

#df = df.reset_index()
df.reset_index().plot.scatter(x = 'voten', y = 'erstunterzeichner', hover_data = ['_name', 'partei'], trendline="ols", width = 800, height = 800)
